In [1]:
# Importing Libraries

#linear algebra and data handling
import pandas as pd
import numpy as np

#preprocessing
from sklearn.preprocessing import StandardScaler

#Developing models
from lightgbm.sklearn import LGBMClassifier
from sklearn.pipeline import Pipeline
from scipy.stats import iqr, randint, uniform

#Evaluating models
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV

# handling system operations
import os

# handling hdf5 files
import h5py

# reading data from google drive
from google.colab import drive

In [2]:
# Owing to the size of the files, the data is to be read from the google drive linked below:
# https://drive.google.com/drive/folders/1m6o2mKnRBaky11jZ7HKtbh1zh6RCBVOk?usp=drive_link

In [3]:
# mounting google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# data_path should store the path to the directory containing all the data files
# as provided in the google drive link above
data_path = '/content/drive/My Drive/Nuclear fusion/'

In [5]:
#Function to list all files within a folder recursively
def list_files_recursive(root_dir):
    all_files = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            all_files.append(os.path.join(dirpath, filename))

    return all_files

In [6]:
# Specifying the path that contains the data for the different tokamak
# types.
# if you are using the same list of data files (names unchanged)
# available in the google drive link, you can safely ignore the next
# 4 comment lines
# For the J-TEXT, ensure the folder name contains "J-TEXt"
# For CMOD (both train and evaluation sets), ensure the folder name
# contains "CMod"
# For HL-2A, ensure the folder name contains "JDDB"
jtext_path = data_path + 'J-TEXT processed_data_1k_5k_final'
cmod_path = data_path + 'CMod_evaluate_20_10_2023'
cmodtr_path = data_path + 'CMod_train'
h2la_path = data_path + 'JDDB_repo_2A_5k'

# storing the list of shot files available for each tokamak type
h2la_list = list_files_recursive(h2la_path)
h2la_list.sort()
cmodtr_list = list_files_recursive(cmodtr_path)
cmodtr_list.sort()
cmod_list = list_files_recursive(cmod_path)
cmod_list.sort()
jtext_list = list_files_recursive(jtext_path)
jtext_list.sort()

In [7]:
# confirming the required length of submission file
len(cmod_list)

413

In [8]:
# function to take in list of hdf5 files of the shots and create features
# out of each shot and finally return a dataframe of features and an array
# of labels
def load_hdf5_dataset(file_list, key):
    data_list = []
    label_list = []

    # for each file within the file list passed to the function
    for filename in file_list:
        with h5py.File(filename, 'r') as f:

          # if the channel/diagnostic signal passed as the 'key' argument
          # is present in that particular shot file
          if key in f["data"]:

            # And the list of entries for the diagnostic signal is greater
            # than or equal to 60. [This is the case because 60 was the least
            # number of entries for the chosen signal in the test data that
            # this model is going to be tested on.]
            if len(f["data"][key]) >= 60:

                # Store the list of entries for the signal in that shot file
                data_array = f["data"][key][:]

                # create another list that stores the single-step change in
                #values for the entries in a particular signal
                diff = data_array[1:]-data_array[:-1]

                # engineer features out of the two lists above
                data_arrays_p = np.concatenate((
                                                 np.array([len(data_array)]),
                                                 np.array([np.mean(data_array)]),
                                                 np.array([np.ptp(data_array)]),
                                                 np.array([np.std(data_array)]),
                                                 np.array([np.max(data_array)]),
                                                 np.array([np.min(data_array)]),
                                                 np.array([np.median(data_array)]),
                                                 np.array([np.var(data_array)]),
                                                 np.array([iqr(data_array)]),
                                                 np.array([np.max(diff)]),
                                                 np.array([np.min(diff)]),
                                                 np.array([np.mean(diff)]),
                                                 np.array([np.std(diff)]),
                                                 np.array([np.ptp(diff)]),
                                                 np.array([np.median(diff)]),
                                                 np.array([np.var(diff)]),
                                                 np.array([iqr(diff)]),
                                                 np.array([f["meta"][key][()]])
                                                 ))

                #encoding tokamak type
                if 'CMod' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([0,0,1])
                                                ))
                elif 'J-TEXT' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([0,1,0])
                                                ))
                elif 'JDDB' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([1,0,0])
                                                ))

                data_list.append(data_arrays_p)

                # Reading the label
                label = f["meta"]["IsDisrupt"][()]
                label_list.append(label)

    return pd.DataFrame(data_list), np.array(label_list)

In [9]:
# function to take in list of hdf5 files of the shots and create features
# out of each shot and finally return a dataframe of features on which predictions
# can be made for the test data
def load_test(file_list, key):
    data_list = []

    # for each file within the file list passed to the function
    for filename in file_list:
        with h5py.File(filename, 'r') as f:

          # if the channel/diagnostic signal passed as the 'key' argument
          # is present in that particular shot file
          if key in f["data"]:

            # And the list of entries for the diagnostic signal is greater
            # than or equal to 60. [This is the case because 60 was the least
            # number of entries for the chosen signal in the test data that
            # this model is going to be tested on.]
            if len(f["data"][key]) >= 60:

                # Store the list of entries for the signal in that shot file
                data_array = f["data"][key][:]

                # create another list that stores the single-step change in
                #values for the entries in a particular signal
                diff = data_array[1:]-data_array[:-1]

                # engineer features out of the two lists above
                data_arrays_p = np.concatenate((
                                                 np.array([len(data_array)]),
                                                 np.array([np.mean(data_array)]),
                                                 np.array([np.ptp(data_array)]),
                                                 np.array([np.std(data_array)]),
                                                 np.array([np.max(data_array)]),
                                                 np.array([np.min(data_array)]),
                                                 np.array([np.median(data_array)]),
                                                 np.array([np.var(data_array)]),
                                                 np.array([iqr(data_array)]),
                                                 np.array([np.max(diff)]),
                                                 np.array([np.min(diff)]),
                                                 np.array([np.mean(diff)]),
                                                 np.array([np.std(diff)]),
                                                 np.array([np.ptp(diff)]),
                                                 np.array([np.median(diff)]),
                                                 np.array([np.var(diff)]),
                                                 np.array([iqr(diff)]),
                                                 np.array([f["meta"][key][()]])
                                                 ))

                #encoding tokamak type
                if 'CMod' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([0,0,1])
                                                ))
                elif 'J-TEXT' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([0,1,0])
                                                ))
                elif 'JDDB' in filename:
                  data_arrays_p = np.concatenate((data_arrays_p,np.array([1,0,0])
                                                ))

                data_list.append(data_arrays_p)


    return pd.DataFrame(data_list)

In [10]:
# load data create dataframe for predicting on submission test data
data_test = load_test(cmod_list,key='.tci.results:nl_04')

In [11]:
# load J-TEXT data as dataframe with features and labels
data_j, labels_j = load_hdf5_dataset(jtext_list,key='polaris_den_v09')
data_j['Is_disrupt'] = labels_j

In [12]:
# load CMod evaluation data as dataframe with features and labels.
# This will be used as validation data
data_c, labels_c = load_hdf5_dataset(cmodtr_list,key='.tci.results:nl_04')
data_c['Is_disrupt'] = labels_c

In [13]:
# load HL-2A data as dataframe with features and labels
data_h, labels_h = load_hdf5_dataset(h2la_list,key='CCO-DF:DENSITY1')
data_h['Is_disrupt'] = labels_h


In [14]:
# combine HL-2A and J-TEXT data as train dataframe
train = pd.concat([data_h,data_j],axis=0).reset_index(drop=True)
train.columns = train.columns.astype('str')

In [15]:
# renaming columns to have names of the same type (string)
data_test.columns = data_test.columns.astype('str')

In [16]:
# renaming columns to have names of the same type (string)
data_c.columns = data_c.columns.astype('str')

In [17]:
# ensure reproducibility for all random processes (particularly scipy
# uniform and randint methods)
np.random.seed(5)

In [18]:
# Defining function that will run the hyperparameter tuning
def runmodel(model,tuning_params,scorer=make_scorer(f1_score),n_iter=5):
  pipe = Pipeline(steps=[
        ('sc',StandardScaler()),
          ('classifier',model)
        ])
  r_search = RandomizedSearchCV(pipe,tuning_params,n_jobs=-1,verbose=-1,
                                scoring=scorer,cv=10,n_iter=n_iter,
                                random_state=2,error_score='raise')
  r_search.fit(train.drop('Is_disrupt',axis=1),train['Is_disrupt'])
  return r_search

# LGBMClassifier

In [19]:
# creating an instance of LGBMClassifier to be used for randomizedsearchcv
# tuning process
model=LGBMClassifier(n_jobs=-1,random_state=20,verbose=1)

# Define the parameter distribution space for RandomizedSearchCV
tuning_params = {
    'classifier__boosting_type': ['gbdt','dart'],
    'classifier__learning_rate': uniform(loc=0.001, scale=1.0),  # Values between 0.001 and 1.0
    'classifier__n_estimators': randint(50, 501),  # Values between 50 and 500
    'classifier__num_leaves': randint(10, 201),  # Values between 10 and 200
    'classifier__max_depth': randint(-1, 51),  # -1 represents no limit, values between -1 and 50
    'classifier__min_child_samples': randint(5, 51),  # Values between 5 and 50
    'classifier__reg_alpha': uniform(loc=0.0, scale=1.0),  # Values between 0 and 1
    'classifier__reg_lambda': uniform(loc=0.0, scale=1.0),  # Values between 0 and 1
    'classifier__min_child_weight': uniform(loc=0.001, scale=10.0),  # Values between 0.001 and 10
    'classifier__min_split_gain': uniform(loc=0.0, scale=0.2),  # Values between 0 and 0.2
    'classifier__bagging_freq': randint(0, 6),  # Values between 0 and 5
    'classifier__bagging_fraction': uniform(loc=0.6, scale=0.4),  # Values between 0.6 and 1.0
    'classifier__feature_fraction': uniform(loc=0.6, scale=0.4),  # Values between 0.6 and 1.0
    'classifier__path_smooth': uniform(loc=0.0, scale=1.0),  # Values between 0 and 1
}

In [20]:
# fitting the model to the data
#lgbm= runmodel(model,tuning_params,n_iter=60)  # search process is time-consuming so it is commented out

In [21]:
# extracting the optimum parameters
#lgbm.best_params_ # Also commented out since the randomizedsearchcv process above has been commented out

In [22]:
# Results of tuning hyperparameter over a parameter search space using
# randomizedsearchcv of 60 iterations
params = {'bagging_fraction': 0.8831884682475684,
 'bagging_freq': 4,
 'boosting_type': 'dart',
 'feature_fraction': 0.9108411997878578,
 'learning_rate': 0.5037787525092214,
 'max_depth': 25,
 'min_child_samples': 50,
 'min_child_weight': 6.801272575129432,
 'min_split_gain': 0.13542794845755993,
 'n_estimators': 472,
 'num_leaves': 137,
 'path_smooth': 0.4144600551329115,
 'reg_alpha': 0.8486845142673234,
 'reg_lambda': 0.4445709417507092}

In [23]:
# Creating the pipeline based on the obtained hyperparameters
final_pipe = Pipeline(steps=[
        ('sc',StandardScaler()),
          ('classifier',LGBMClassifier(**params,n_jobs=-1,
                                       random_state=20,verbose=1))
        ])

In [24]:
# Fitting the pipeline steps to the data
final_pipe.fit(train.drop('Is_disrupt',axis=1),train['Is_disrupt'])

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9108411997878578, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9108411997878578
[LightGBM] [Warning] bagging_fraction is set=0.8831884682475684, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8831884682475684
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9108411997878578, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9108411997878578
[LightGBM] [Warning] bagging_fraction is set=0.8831884682475684, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8831884682475684
[LightGBM] [Info] Number of positive: 1156, number of negative: 1889
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000734 seconds.
You can s

Pipeline(steps=[('sc', StandardScaler()),
                ('classifier',
                 LGBMClassifier(bagging_fraction=0.8831884682475684,
                                bagging_freq=4, boosting_type='dart',
                                feature_fraction=0.9108411997878578,
                                learning_rate=0.5037787525092214, max_depth=25,
                                min_child_samples=50,
                                min_child_weight=6.801272575129432,
                                min_split_gain=0.13542794845755993,
                                n_estimators=472, n_jobs=-1, num_leaves=137,
                                path_smooth=0.4144600551329115, random_state=20,
                                reg_alpha=0.8486845142673234,
                                reg_lambda=0.4445709417507092, verbose=1))])

In [25]:
# obtaining the train f1 score
lgbmtrain = f1_score(train['Is_disrupt'],final_pipe.predict(train.drop('Is_disrupt',axis=1)))
lgbmtrain

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9108411997878578, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9108411997878578
[LightGBM] [Warning] bagging_fraction is set=0.8831884682475684, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8831884682475684


0.990484429065744

In [26]:
# obtaining the test f1 score
lgbmtest = f1_score(data_c['Is_disrupt'],final_pipe.predict(data_c.drop('Is_disrupt',axis=1)))
lgbmtest

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9108411997878578, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9108411997878578
[LightGBM] [Warning] bagging_fraction is set=0.8831884682475684, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8831884682475684


1.0

In [27]:
# predicting on the submission data
prediction = final_pipe.predict(data_test)

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.9108411997878578, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9108411997878578
[LightGBM] [Warning] bagging_fraction is set=0.8831884682475684, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8831884682475684


In [28]:
# creating the correct ID values for the submission predictions
id = ['ID_'+i[-15:-5] for i in cmod_list ]

In [29]:
# Confirming that the length of predictions are as expected based on the
# the number of observations in the submission set
len(prediction)==len(id)==413

True

In [30]:
# creating a submission dataframe
submission = pd.DataFrame({'Shot_list':id,'Is_disrupt':prediction})
submission

,Shot_list,Is_disrupt
0,ID_1120214004,1
1,ID_1120214013,1
2,ID_1120216004,1
3,ID_1120222021,0
4,ID_1120223024,1
...,...,...
408,ID_1160927016,0
409,ID_1160928018,0
410,ID_1160929014,0
411,ID_1160929021,0


In [31]:
# A look at the distribution of predictions in the submission set
submission['Is_disrupt'].value_counts()

0    273
1    140
Name: Is_disrupt, dtype: int64

In [32]:
# Exporting the submission dataframe to a csv file
submission.to_csv('Submission.csv',index=False)